## Manual feature labelling and prompt building

### Setup

In [ ]:
# Imports

import os
import torch as t
from nnsight import LanguageModel
import datasets
import anthropic
from tqdm import tqdm
import re
import ast
import pickle
from collections import defaultdict
from circuitsvis.activations import text_neuron_activations
from transformers import AutoTokenizer
import random
import json

import experiments.utils as utils
from experiments.autointerp import (
    get_max_activating_prompts,
    highlight_top_activations,
    compute_dla,
    format_examples,
    evaluate_binary_llm_output,
    get_autointerp_inputs_for_all_saes,
)
from experiments.explainers.simple.prompt_builder import build_prompt
from experiments.explainers.simple.prompts import build_system_prompt

DEBUGGING = True

if DEBUGGING:
    tracer_kwargs = dict(scan=True, validate=True)
else:
    tracer_kwargs = dict(scan=False, validate=False)

%load_ext autoreload
%autoreload 2

In [ ]:
profession_dict = {
    "accountant": 0,
    "architect": 1,
    "attorney": 2,
    "chiropractor": 3,
    "comedian": 4,
    "composer": 5,
    "dentist": 6,
    "dietitian": 7,
    "dj": 8,
    "filmmaker": 9,
    "interior_designer": 10,
    "journalist": 11,
    "model": 12,
    "nurse": 13,
    "painter": 14,
    "paralegal": 15,
    "pastor": 16,
    "personal_trainer": 17,
    "photographer": 18,
    "physician": 19,
    "poet": 20,
    "professor": 21,
    "psychologist": 22,
    "rapper": 23,
    "software_engineer": 24,
    "surgeon": 25,
    "teacher": 26,
    "yoga_teacher": 27,
    "male / female": "male / female",
    "professor / nurse": "professor / nurse",
    "male_professor / female_nurse": "male_professor / female_nurse",
    "biased_male / biased_female": "biased_male / biased_female",
}

In [ ]:
# Load dictionary

dictionaries_path = "../dictionary_learning/dictionaries/autointerp_test_data"

# Current recommended way to generate graphs. You can copy paste ae_sweep_paths directly from bib_intervention.py
ae_sweep_paths = {
    "pythia70m_sweep_topk_ctx128_0730": {"resid_post_layer_3": {"trainer_ids": [2, 6, 10, 18]}},
    "gemma-2-2b_test_sae": {"resid_post_layer_12": {"trainer_ids": [2]}},
}
pythia_sweep_name = list(ae_sweep_paths.keys())[0]
pythia_submodule_trainers = ae_sweep_paths[pythia_sweep_name]

pythia_group_paths = utils.get_ae_group_paths(
    dictionaries_path, pythia_sweep_name, pythia_submodule_trainers
)
pythia_ae_paths = utils.get_ae_paths(pythia_group_paths)

gemma_sweep_name = list(ae_sweep_paths.keys())[1]
gemma_submodule_trainers = ae_sweep_paths[gemma_sweep_name]

gemma_group_paths = utils.get_ae_group_paths(
    dictionaries_path, gemma_sweep_name, gemma_submodule_trainers
)
gemma_ae_paths = utils.get_ae_paths(gemma_group_paths)

chosen_class_indices = [
    "male / female",
    "professor / nurse",
    "male_professor / female_nurse",
    "biased_male / biased_female",
    "accountant",
    "architect",
    "attorney",
    "dentist",
    "filmmaker",
]

In [ ]:
sampled_indices = [0, 10, 100, 1000]
class_indices_1 = ["male / female", "professor / nurse", "accountant", "architect"]
class_indices_2 = ["male_professor / female_nurse", "attorney", "dentist", "filmmaker"]
class_indices_3 = class_indices_1 + class_indices_2

combinations = [
    {"sae": pythia_ae_paths[0], "class_indices": class_indices_1},
    {"sae": pythia_ae_paths[1], "class_indices": class_indices_2},
    {"sae": pythia_ae_paths[2], "class_indices": class_indices_1},
    {"sae": pythia_ae_paths[3], "class_indices": class_indices_2},
    {"sae": gemma_ae_paths[0], "class_indices": class_indices_3},
]


def generate_combinations(sampled_indices, combinations):
    result = []
    for combo in combinations:
        sae = combo["sae"]
        class_indices = combo["class_indices"]
        for class_index in class_indices:
            for sampled_index in sampled_indices:
                result.append(
                    {"sae": sae, "class_index": class_index, "sampled_index": sampled_index}
                )
    return result


orig_combinations_list = generate_combinations(sampled_indices, combinations)
random.seed(1776)
combinations_list = orig_combinations_list.copy()
random.shuffle(combinations_list)
print(f"There are {len(combinations_list)} combinations to evaluate.")

In [ ]:
print(combinations_list[2])
assert combinations_list[0] == {'sae': '../dictionary_learning/dictionaries/autointerp_test_data/pythia70m_sweep_topk_ctx128_0730/resid_post_layer_3/trainer_6', 'class_index': 'dentist', 'sampled_index': 100}
assert combinations_list[2] == {'sae': '../dictionary_learning/dictionaries/autointerp_test_data/pythia70m_sweep_topk_ctx128_0730/resid_post_layer_3/trainer_10', 'class_index': 'male / female', 'sampled_index': 1000}

In [ ]:
pythia_model_eval_config = utils.ModelEvalConfig.from_sweep_name(pythia_sweep_name)
pythia_model_name = pythia_model_eval_config.full_model_name
pythia_tokenizer = AutoTokenizer.from_pretrained(pythia_model_name)

gemma_model_eval_config = utils.ModelEvalConfig.from_sweep_name(gemma_sweep_name)
gemma_model_name = gemma_model_eval_config.full_model_name
gemma_tokenizer = AutoTokenizer.from_pretrained(gemma_model_name)

In [ ]:
# Run max activating examples once from all SAEs

k_inputs_per_feature = 10
example_ae_path = pythia_ae_paths[0]

with open(os.path.join(example_ae_path, "max_activating_inputs.pkl"), "rb") as f:
    max_activating_inputs = pickle.load(f)

max_token_idxs_FKL = max_activating_inputs["max_tokens_FKL"]
max_activations_FKL = max_activating_inputs["max_activations_FKL"]
top_dla_token_idxs_FK = max_activating_inputs["dla_results_FK"]

### Select class

In [ ]:
### Find features relevant to a profession from the class_probe node_effects.pkl

class_name = "male / female"
k_features_per_concept = 3

filename_counter = ""
class_id = profession_dict[class_name]
node_effects_filename = f"{example_ae_path}/node_effects{filename_counter}.pkl"

with open(node_effects_filename, "rb") as f:
    node_effects = pickle.load(f)

effects = node_effects[class_id]
print(effects.shape)


top_k_values, top_k_indices = t.topk(effects, k_features_per_concept)
t.set_printoptions(sci_mode=False)
print(top_k_values)
print(top_k_indices)

selected_token_idxs_FKL = max_token_idxs_FKL[top_k_indices]
selected_activations_FKL = max_activations_FKL[top_k_indices]
top_dla_token_idxs_FK = top_dla_token_idxs_FK[top_k_indices]

In [ ]:
# Format max_activating_inputs by << emphasizing>> max act examples

num_top_emphasized_tokens = 5

print(top_dla_token_idxs_FK.shape)

example_prompts = format_examples(
    pythia_tokenizer, selected_token_idxs_FKL, selected_activations_FKL, num_top_emphasized_tokens
)

top_dla_tokens_list = []

tokens_list = utils.list_decode(top_dla_token_idxs_FK, pythia_tokenizer)
tokens = tokens_list[0]

print(",".join(tokens))


In [ ]:
import einops

# Checking for features that didn't activate on at least displayed_prompts inputs

displayed_prompts = 10
for current_combination in combinations_list:
    ae_path = current_combination["sae"]
    class_index = current_combination["class_index"]
    sampled_index = current_combination["sampled_index"]

    class_id = profession_dict[class_index]

    if "pythia" in ae_path:
        tokenizer = pythia_tokenizer
    else:
        tokenizer = gemma_tokenizer

    inputs_path = os.path.join(ae_path, "max_activating_inputs.pkl")
    node_effects_path = os.path.join(ae_path, "node_effects.pkl")

    with open(inputs_path, "rb") as f:
        max_activating_inputs = pickle.load(f)

    with open(node_effects_path, "rb") as f:
        node_effects = pickle.load(f)

    max_token_idxs_FKL = max_activating_inputs["max_tokens_FKL"]
    max_activations_FKL = max_activating_inputs["max_activations_FKL"]
    top_dla_token_idxs_FK = max_activating_inputs["dla_results_FK"]

    effects = node_effects[class_id]

    top_k_values, top_k_indices = t.topk(effects, 2000)

    sae_feat_index = top_k_indices[sampled_index]

    selected_token_idxs_1KL = max_token_idxs_FKL[sae_feat_index, :displayed_prompts, :].unsqueeze(0)
    selected_activations_1KL = max_activations_FKL[sae_feat_index, :displayed_prompts, :].unsqueeze(
        0
    )
    top_dla_token_idxs_K = top_dla_token_idxs_FK[sae_feat_index]

    selected_activations_K = einops.reduce(selected_activations_1KL, "1 K L -> K", "sum")
    selected_activations_nonzero = selected_activations_K[selected_activations_K != 0]

    if selected_activations_nonzero.shape != selected_activations_K.shape:
        print(
            f"Warning: {selected_activations_K.shape} activations, {selected_activations_nonzero.shape} non-zero activations"
        )

## MANUAL LABELING BEGINS HERE

Run manual inputs creation to view DLA and top 4 example prompts.

Add scores / chain of thought to Label adding per example.

Save as json once done.

In [ ]:
manual_labels = {}

In [ ]:
import json

with open("manual_labels.json", "r") as f:
    manual_labels = json.load(f)

In [ ]:
current_idx = 7

displayed_prompts = 10
num_top_emphasized_tokens = 5
include_activations = True
t.set_printoptions(sci_mode=False)

In [ ]:
# Manual inputs creation

current_combination = combinations_list[current_idx]
ae_path = current_combination["sae"]
class_index = current_combination["class_index"]
sampled_index = current_combination["sampled_index"]

class_id = profession_dict[class_index]

if "pythia" in ae_path:
    tokenizer = pythia_tokenizer
else:
    tokenizer = gemma_tokenizer

print(f"current idx: {current_idx}")
print(f"SAE path: {ae_path}")
print(f"class index: {class_index}")
print(f"sampled index: {sampled_index}")

inputs_path = os.path.join(ae_path, "max_activating_inputs.pkl")
node_effects_path = os.path.join(ae_path, "node_effects.pkl")

with open(inputs_path, "rb") as f:
    max_activating_inputs = pickle.load(f)

with open(node_effects_path, "rb") as f:
    node_effects = pickle.load(f)

max_token_idxs_FKL = max_activating_inputs["max_tokens_FKL"]
max_activations_FKL = max_activating_inputs["max_activations_FKL"]
top_dla_token_idxs_FK = max_activating_inputs["dla_results_FK"]


effects = node_effects[class_id]

top_k_values, top_k_indices = t.topk(effects, 2000)

sae_feat_index = top_k_indices[sampled_index]

selected_token_idxs_1KL = max_token_idxs_FKL[sae_feat_index, :displayed_prompts, :].unsqueeze(0)
selected_activations_1KL = max_activations_FKL[sae_feat_index, :displayed_prompts, :].unsqueeze(0)
top_dla_token_idxs_K = top_dla_token_idxs_FK[sae_feat_index]

example_prompts = format_examples(
    tokenizer,
    selected_token_idxs_1KL,
    selected_activations_1KL,
    num_top_emphasized_tokens,
    include_activations,
)
tokens_list = utils.list_decode(top_dla_token_idxs_K, tokenizer)
tokens_string = ",".join(tokens_list)

# Show activations in HTML
selected_token_strs_1KL = utils.list_decode(selected_token_idxs_1KL, tokenizer)
highlighted_token_idxs_1KL = t.topk(selected_activations_1KL, num_top_emphasized_tokens, dim=-1).indices

highlighted_token_values_1KL = t.zeros_like(selected_activations_1KL)
highlighted_token_values_1KL.scatter_(-1, highlighted_token_idxs_1KL, selected_activations_1KL.gather(-1, highlighted_token_idxs_1KL))

selected_activations_KL11 = [highlighted_token_values_1KL[0, k, :, None, None] for k in range(k_inputs_per_feature)]
html_activations = text_neuron_activations(selected_token_strs_1KL[0], selected_activations_KL11)

print(tokens_string)
# print(example_prompts[0])
display(html_activations)

previous_idx = current_idx
current_idx += 2

In [ ]:
if str(previous_idx) in manual_labels:
    print(f"per_class_scores: {manual_labels[str(previous_idx)]['per_class_scores']}")
    print(f"chain_of_thought: {manual_labels[str(previous_idx)]['chain_of_thought']}")

In [ ]:
# Label adding

per_class_scores = {
    "male / female": 0,
    "professor / nurse": 0,
    "male_professor / female_nurse": 0,
    "accountant": 1,
    "architect": 0,
    "attorney": 1,
    "dentist": 0,
    "filmmaker": 0,
}

chain_of_thought = """The top promoted logits don't have an obvious pattern.
The activating inputs are primarily on words like club, service, etc.
This possibly relates to attornies and accountants.
I will rate both classes as a 1."""

# chain_of_thought = """The top promoted logits don't have an obvious pattern.
# The activating inputs are primarily on spaces.
# I don't see an obvious pattern.
# I will rate all classes as a 0."""

new_label = {
    "sae": ae_path,
    "class_index": class_index,
    "sampled_index": sampled_index,
    "sae_feat_index": sae_feat_index.item(),
    "example_prompts": example_prompts,
    "tokens_string": tokens_string,
    "per_class_scores": per_class_scores,
    "chain_of_thought": chain_of_thought,
}

manual_labels[previous_idx] = new_label

print(f"Add the following idx to manual_labels: {previous_idx}")

In [ ]:
print(manual_labels.keys())

import json


def update_json_file(filename, new_data):
    try:
        # Read existing data
        with open(filename, "r") as f:
            data = json.load(f)
    except FileNotFoundError:
        # If file doesn't exist, start with an empty dictionary
        data = {}

    # Convert all keys to strings (if they aren't already)
    data = {str(k): v for k, v in data.items()}
    new_data = {str(k): v for k, v in new_data.items()}

    print(f"Existing keys: {data.keys()}")
    print(f"New keys: {new_data.keys()}")

    # Update data with new_data (this overwrites existing keys)
    data.update(new_data)

    # Write updated data back to file
    with open(filename, "w") as f:
        json.dump(data, f, indent=2)


filename = "manual_labels.json"
update_json_file(filename, manual_labels)

In [ ]:
import json
import matplotlib.pyplot as plt

# Load the JSON files
with open('manual_labels.json', 'r') as f:
    manual_labels = json.load(f)

with open('manual_labels_can.json', 'r') as f:
    manual_labels_can = json.load(f)

# Function to create a bar chart for a single key
def create_chart(key):
    labels = manual_labels[key]['per_class_scores'].keys()
    manual_scores = [manual_labels[key]['per_class_scores'][label] for label in labels]
    if key not in manual_labels_can:
        print(f"Key {key} not found in manual_labels_can.")
        return
    can_scores = [manual_labels_can[key]['per_class_scores'][label] for label in labels]

    x = range(len(labels))
    width = 0.35

    fig, ax = plt.subplots(figsize=(12, 6))
    ax.bar([i - width/2 for i in x], manual_scores, width, label='manual_labels')
    ax.bar([i + width/2 for i in x], can_scores, width, label='manual_labels_can')

    ax.set_ylabel('Scores')
    ax.set_title(f'Comparison of Scores for Key {key}')
    ax.set_xticks(x)
    ax.set_xticklabels(labels, rotation=45, ha='right')
    ax.legend()

    plt.tight_layout()
    # plt.savefig(f'comparison_chart_key_{key}.png')
    plt.show()
    plt.close()

# Create charts for all keys
for key in manual_labels.keys():
    create_chart(key)

print("All charts have been generated and saved as PNG files.")